<a href="https://colab.research.google.com/github/alfanadhya/acne-detection-with-glcm-and-gabor/blob/main/%5Barr%5D_get_glcm_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# LIBRARIES

# to get GLCM features
from skimage.feature import greycomatrix, greycoprops

# for feature normalization
from sklearn import preprocessing

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# to get file directory
import glob
from natsort import natsorted
from os.path import join

# to save scaler
import joblib

In [ ]:
# DATA
acne_blocks_path = '/content/drive/MyDrive/DatasetTA_Playground/acne_blocks/'
no_acne_blocks_path = '/content/drive/MyDrive/DatasetTA_Playground/skin_blocks/'

acne_blocks_dir = natsorted(glob.glob(join(acne_blocks_path, '*.jpg')))
no_acne_blocks_dir = natsorted(glob.glob(join(no_acne_blocks_path, '*.jpg')))

In [ ]:
acne_blocks = []

for block in acne_blocks_dir:
  acne_img = cv2.imread(block)
  acne_blocks.append(cv2.cvtColor(acne_img, cv2.COLOR_BGR2GRAY))

no_acne_blocks = []

for block in no_acne_blocks_dir:
  no_acne_img = cv2.imread(block)
  no_acne_blocks.append(cv2.cvtColor(no_acne_img, cv2.COLOR_BGR2GRAY))

In [ ]:
feats_name = ['contrast', 'correlation', 'energy', 'homogeneity']
feats = np.zeros((len(no_acne_blocks)+len(acne_blocks), (len(feats_name)*4)+1))
degrees = [0, np.pi/4, np.pi/2, 3*np.pi/4]

for counter, block in enumerate(no_acne_blocks):
  g = greycomatrix(block, [1], degrees, normed=True, symmetric=True)
  temp = []
  for c, feat in enumerate(feats_name):
      preps = greycoprops(g, feats_name[c])
      preps = np.concatenate((preps))
      temp.append(preps)
  temp = np.concatenate(temp)
  feats[counter, :feats.shape[1]-1] = temp
  feats[counter, -1] = 0

for counter, block in enumerate(acne_blocks):
  g = greycomatrix(block, [1], degrees, normed=True, symmetric=True)
  idx = len(no_acne_blocks)+counter
  temp = []
  for c, feat in enumerate(feats_name):
      preps = greycoprops(g, feats_name[c])
      preps = np.concatenate((preps))
      temp.append(preps)
  temp = np.concatenate(temp)
  feats[idx, :feats.shape[1]-1] = temp
  feats[idx, -1] = 1

In [ ]:
'''feats_name = ['contrast', 'dissimilarity', 'correlation', 'energy', 'homogeneity', 'ASM', 'label']
feats = np.zeros((len(no_acne_blocks)+len(acne_blocks), len(feats_name)))

for counter, block in enumerate(no_acne_blocks):
  g = greycomatrix(block, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], normed=True, symmetric=True)
  feats[counter, 0] = np.mean(greycoprops(g, feats_name[0]))
  feats[counter, 1] = np.mean(greycoprops(g, feats_name[1]))
  feats[counter, 2] = np.mean(greycoprops(g, feats_name[2]))
  feats[counter, 3] = np.mean(greycoprops(g, feats_name[3]))
  feats[counter, 4] = np.mean(greycoprops(g, feats_name[4]))
  feats[counter, 5] = np.mean(greycoprops(g, feats_name[5]))
  feats[counter, 6] = 0

for counter, block in enumerate(acne_blocks):
  g = greycomatrix(block, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], normed=True, symmetric=True)
  idx = len(no_acne_blocks)+counter
  feats[idx, 0] = np.mean(greycoprops(g, feats_name[0]))
  feats[idx, 1] = np.mean(greycoprops(g, feats_name[1]))
  feats[idx, 2] = np.mean(greycoprops(g, feats_name[2]))
  feats[idx, 3] = np.mean(greycoprops(g, feats_name[3]))
  feats[idx, 4] = np.mean(greycoprops(g, feats_name[4]))
  feats[idx, 5] = np.mean(greycoprops(g, feats_name[5]))
  feats[idx, 6] = 1'''

In [ ]:
# feats_df = pd.DataFrame(feats, columns = feats_name)
feats_df = pd.DataFrame(feats)
feats_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,40.215789,53.188366,31.210526,42.739612,0.811933,0.749405,0.862314,0.795476,0.058104,0.056227,0.058873,0.056533,0.198383,0.162813,0.230666,0.170402,0.0
1,48.781579,43.911357,41.310526,57.481994,0.878568,0.885687,0.900225,0.852542,0.048202,0.050015,0.049232,0.049281,0.148067,0.190980,0.192320,0.158694,0.0
2,16.950000,11.886427,9.505263,29.027701,0.969752,0.977934,0.983511,0.946887,0.053155,0.055019,0.058104,0.048929,0.256120,0.278518,0.313447,0.177024,0.0
3,111.510526,114.160665,51.931579,84.318560,0.915331,0.912588,0.961496,0.934826,0.041734,0.041597,0.043241,0.043579,0.135180,0.108945,0.161223,0.134038,0.0
4,66.886842,66.675900,71.471053,69.110803,0.650998,0.645129,0.624250,0.626339,0.052368,0.052959,0.051602,0.053067,0.163965,0.170956,0.165181,0.147205,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773,51.261905,64.777778,42.261905,69.166667,0.382747,0.203960,0.482038,0.130114,0.131492,0.125769,0.127108,0.131762,0.167509,0.101771,0.228760,0.207806,1.0
1774,74.097222,77.727273,46.051948,63.863636,0.235527,0.241756,0.548968,0.373911,0.108476,0.109259,0.107094,0.104425,0.165262,0.119371,0.231398,0.157970,1.0
1775,42.250000,66.805195,22.318182,44.961039,0.467779,0.210823,0.726918,0.467336,0.096348,0.095875,0.097754,0.092746,0.119872,0.155000,0.289280,0.141790,1.0
1776,48.533333,59.054545,17.378788,47.763636,0.271976,0.018559,0.715031,0.259623,0.113039,0.126622,0.112876,0.112080,0.151337,0.244123,0.252560,0.176283,1.0


In [ ]:
# Create scaler
scaler = preprocessing.MinMaxScaler()
names = feats_df.columns[:-1]

# Normalization
d = scaler.fit_transform(feats_df.iloc[:,:-1])

# Move to new dataframe
scaled_df = pd.DataFrame(d, columns=names)
scaled_df['label'] = feats_df.iloc[:,-1]

In [ ]:
scaled_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,label
0,0.019547,0.021166,0.015900,0.020335,0.880986,0.887441,0.938338,0.908744,0.080339,0.077562,0.022167,0.077612,0.200082,0.413979,0.215516,0.463075,0.0
1,0.023627,0.017687,0.021509,0.027453,0.917330,0.943773,0.950871,0.928649,0.066626,0.071809,0.012352,0.066424,0.154317,0.505139,0.178407,0.381589,0.0
2,0.009952,0.005128,0.006138,0.018981,0.984239,0.992717,0.993687,0.977913,0.070367,0.076867,0.017952,0.066885,0.230196,0.688172,0.288606,0.329245,0.0
3,0.055950,0.048178,0.026818,0.040647,0.949703,0.962269,0.984266,0.973317,0.058903,0.059282,0.005643,0.061130,0.118623,0.237995,0.138774,0.358652,0.0
4,0.032270,0.026332,0.036118,0.032076,0.762883,0.829011,0.821276,0.824028,0.074153,0.073915,0.015706,0.075507,0.161625,0.424806,0.150937,0.366979,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773,0.024638,0.025349,0.021387,0.032569,0.561368,0.601980,0.741019,0.565057,0.185958,0.177865,0.093148,0.186339,0.167509,0.258655,0.217173,0.518060,1.0
1774,0.034265,0.029029,0.022687,0.029458,0.481567,0.635021,0.770222,0.689193,0.154035,0.153023,0.066904,0.138866,0.195956,0.265199,0.221768,0.311321,1.0
1775,0.021279,0.027235,0.011455,0.021997,0.611623,0.595299,0.863988,0.727444,0.133099,0.141672,0.067674,0.136827,0.118244,0.389981,0.272131,0.367136,1.0
1776,0.023327,0.023109,0.008795,0.022491,0.482652,0.509279,0.857515,0.629811,0.159861,0.179070,0.078363,0.158505,0.151337,0.620450,0.241330,0.439474,1.0


In [ ]:
# Save normalization scaler
joblib.dump(scaler, '/content/drive/MyDrive/DatasetTA_Playground/experiments/knn_models/scaler_grayscale_16.pkl')

# Save normalized features
feats_path = '/content/drive/MyDrive/DatasetTA_Playground/experiments/features/glcm_features_grayscale_16.xlsx'
scaled_df.to_excel(feats_path)